# closed_eyes_dataset을 200*200으로 resize하자!
# test등 분할하기

In [1]:
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator

In [2]:
n_batches = 20

In [3]:
dataset = './dataset'
opened_data_path = 'E:/2023/2023_1_1/comp/dataset/opened_eyes_dataset'
closed_data_path = 'E:/2023/2023_1_1/comp/dataset/closed_eyes_dataset'
train_dir = 'E:/2023/2023_1_1/comp/dataset/train'
test_dir = 'E:/2023/2023_1_1/comp/dataset/test'
validation_dir = 'E:/2023/2023_1_1/comp/dataset/validation'

In [60]:
import os

#이름 다시 설정
file_names = os.listdir(closed_data_path)
print(len(file_names))
i = 1000
for name in file_names:
    src = os.path.join(closed_data_path, name)
    dst = 'closed_eye_'+ str(i) +'.jpg'
    if os.path.isfile(closed_data_path+'/'+dst):
        continue
    os.rename(src, dst)
    i+=1 

0


In [4]:
#model
from tensorflow import keras
from keras import layers
from keras import models

model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape=(200, 200, 3))) #input_shape 지정
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D(2, 2)) 
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2)) 
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))


In [5]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
    optimizer = optimizers.Adam(learning_rate = 1e-4),
    metrics = ['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 128)      0

In [61]:
#import os
#file_names = os.listdir(closed_data_path) #경로를 리스트로 만드나보다
#print(file_names)

[]


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   shear_range=0.7, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=[0.9, 2.0],
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=n_batches,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(200, 200),
    batch_size=n_batches,
    class_mode='binary'
)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(200, 200),
    batch_size=n_batches,
    class_mode='binary'
)

Found 5274 images belonging to 2 classes.
Found 1319 images belonging to 2 classes.
Found 1323 images belonging to 2 classes.


In [8]:
for data_batch, label_batch in train_generator:
  print('배치 데이터 크기:', data_batch.shape)
  print('배치 레이블 크기:', label_batch.shape)
  break

배치 데이터 크기: (20, 200, 200, 3)
배치 레이블 크기: (20,)


In [9]:
callbacks = [tf.keras.callbacks.TensorBoard('./logs_model3')]
history = model.fit(
    x= train_generator, 
    steps_per_epoch=50, #10*4도 가능 len(X_train)//batch_size 최대 224
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50

)

Epoch 1/100
50/50 [==============================] - 112s 2s/step - loss: 0.4366 - accuracy: 0.8410 - val_loss: 0.4298 - val_accuracy: 0.8480
Epoch 2/100
50/50 [==============================] - 114s 2s/step - loss: 0.4350 - accuracy: 0.8360 - val_loss: 0.4250 - val_accuracy: 0.8520
Epoch 3/100
50/50 [==============================] - 102s 2s/step - loss: 0.4525 - accuracy: 0.8190 - val_loss: 0.3934 - val_accuracy: 0.8530
Epoch 4/100
50/50 [==============================] - 114s 2s/step - loss: 0.3770 - accuracy: 0.8680 - val_loss: 0.3973 - val_accuracy: 0.8510
Epoch 5/100
50/50 [==============================] - 113s 2s/step - loss: 0.4063 - accuracy: 0.8510 - val_loss: 0.3907 - val_accuracy: 0.8540
Epoch 6/100
50/50 [==============================] - 108s 2s/step - loss: 0.3959 - accuracy: 0.8451 - val_loss: 0.3686 - val_accuracy: 0.8540
Epoch 7/100
50/50 [==============================] - 108s 2s/step - loss: 0.3256 - accuracy: 0.8830 - val_loss: 0.3409 - val_accuracy: 0.8640
Epoch 

In [1]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

#이미지 전처리 유틸리티 모듈
from keras.preprocessing import image
from keras.utils import load_img, img_to_array, array_to_img
fnames = sorted([os.path.join(train_dir + '/closed', fname) for fname in os.listdir(train_dir + '/closed')])

img_path = fnames[12]
img = load_img(img_path, target_size=(150, 150))

x = img_to_array(img)

#(1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

#flow() 메서드는 랜덤하게 변환된 이미지의 배치를 생성
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [117]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

ImportError: You must install pydot (`pip install pydot`) for model_to_dot to work.